In [1]:
# TODO IMPORTANT: CHANGE THESE TO MAMBA AND REMOVE TORCH INSTALL
%pip install torch --index-url https://download.pytorch.org/whl/cu124
%pip install wandb bitsandbytes peft trl accelerate datasets transformers ipywidgets

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
import wandb
from datasets import load_dataset, get_dataset_config_names, load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
import gc

In [5]:
model_name = '/mnt/d/Models/LLaMA2' # TODO change this to wherever the model is
dataset_name = "/mnt/d/Datasets/" + 'openai/gsm8k' # TODO remove the prepended path

In [6]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alexguha (alexguha-wb). Use `wandb login --relogin` to force relogin


True

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True, # TODO Set me to False on a bigger GPU
)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/aguha/CSE_576/venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:182: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [14]:
configs = get_dataset_config_names(dataset_name)
# dataset = load_dataset(dataset_name, configs[0], split='train')
dataset = load_from_disk(dataset_name) # TODO change this to the above line
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [15]:
def format(example):
    text = 'Question: ' + str(example['question']) + '\n\nAnswer: ' + str(example['answer'])
    return [text]

format(dataset[0])

['Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n\nAnswer: Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72']

In [16]:
peft_params = LoraConfig(
    lora_alpha=32,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True,
)

In [18]:
training_arguments = TrainingArguments(
    output_dir="/mnt/d/Models/LLaMA2/LLaMA2-7b-OpenAI-GSM8k-2",
    num_train_epochs=1,
    per_device_train_batch_size=1, # TODO Up me to whatever big number the GPU can handle
    gradient_accumulation_steps=8, # TODO Down me to 1 if batch size is big
    warmup_steps=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    optim='paged_adamw_8bit',
    report_to="wandb",
    logging_steps=50, # TODO: Down me to so that gradient_accumulation_steps * logging_steps ~= 50
    run_name="llama-2-7b-OpenAI-GSM8k-2",
    max_steps=1 # TODO: Remove after testing
)

In [19]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_arguments,
    peft_config=peft_params,
    formatting_func=format,
    max_seq_length=512, # TODO Up me to 4096 when on bigger GPU
)

/home/aguha/CSE_576/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/aguha/CSE_576/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=1, training_loss=1.800492763519287, metrics={'train_runtime': 627.7055, 'train_samples_per_second': 0.013, 'train_steps_per_second': 0.002, 'total_flos': 162594677587968.0, 'train_loss': 1.800492763519287, 'epoch': 1.0})

In [22]:
torch.cuda.empty_cache()
gc.collect()

475

In [ ]:
trainer.save_model("/mnt/d/Models/LLaMA2/LLaMA2-7b-OpenAI-GSM8k-2")

In [21]:
wandb.finish()

train/epoch,▁
train/global_step,▁
total_flos,162594677587968.0
train/epoch,1
train/global_step,1
train_loss,1.80049
train_runtime,627.7055
train_samples_per_second,0.013
train_steps_per_second,0.002
